In [2]:
from pathlib import Path
from PIL import Image
import torch
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def zero_shot_label(img, labels):
    with torch.no_grad():
        i = preprocess(img).unsqueeze(0).to(device)
        t = clip.tokenize([f"a photo of {lbl}" for lbl in labels]).to(device)
        i_feat = model.encode_image(i); t_feat = model.encode_text(t)
        probs = (i_feat @ t_feat.T).softmax(dim=-1).squeeze(0)
        idx = int(probs.argmax().item())
        return labels[idx], probs.tolist()

def evaluate(path_clean, path_poisoned, concept_C, concept_A):
    pc, pp = Path(path_clean), Path(path_poisoned)

    if pc.is_file() and pp.is_file():
        img_c = Image.open(pc).convert("RGB")
        img_p = Image.open(pp).convert("RGB")

        pred_c, probs_c = zero_shot_label(img_c, [concept_C, concept_A])
        pred_p, probs_p = zero_shot_label(img_p, [concept_C, concept_A])

        return {
            "clean_prediction": concept_A if pred_c != concept_C else concept_C,
            "poisoned_prediction": concept_A if pred_p != concept_C else concept_C,
            "C_A_prob_ratio_clean": probs_c,
            "C_A_prob_ratio_poison": probs_p,
            "odds of " + concept_C + " to " + concept_A + " (clean)": str(int(probs_c[0] / probs_c[1])) + " to 1",
            "odds of " + concept_C + " to " + concept_A + " (poisoned)": str(int(probs_p[0] / probs_p[1])) + " to 1",
            "odds increase (poisoned vs clean)": f"{(probs_c[0] / probs_c[1]) / (probs_p[0] / probs_p[1]):.2f} times",
            "odds increase yes/no": (probs_c[0] / probs_c[1]) > (probs_p[0] / probs_p[1])
        }

In [3]:
from pprint import pprint
import pandas as pd

for i in range(20):
    result = evaluate(f"survey_images/dog{i}_clean.png", f"survey_images/dog{i}_poisoned.png", "a dog", "a cat")
    pprint(f"Image {i} results: {result}\n")
    if i == 0:
        df_results = pd.DataFrame()

    df_results = pd.concat([df_results, pd.DataFrame([result])], ignore_index=True)

    if i == 19:
        df_results.to_csv("dog_results.csv", index=False)

("Image 0 results: {'clean_prediction': 'a dog', 'poisoned_prediction': 'a "
 "dog', 'C_A_prob_ratio_clean': [1.0, 0.0001442432403564453], "
 "'C_A_prob_ratio_poison': [0.97607421875, 0.0236968994140625], 'odds of a dog "
 "to a cat (clean)': '6932 to 1', 'odds of a dog to a cat (poisoned)': '41 to "
 "1', 'odds increase (poisoned vs clean)': '168.31 times', 'odds increase "
 "yes/no': True}\n")
("Image 1 results: {'clean_prediction': 'a dog', 'poisoned_prediction': 'a "
 "dog', 'C_A_prob_ratio_clean': [0.99951171875, 0.0004513263702392578], "
 "'C_A_prob_ratio_poison': [1.0, 0.00017404556274414062], 'odds of a dog to a "
 "cat (clean)': '2214 to 1', 'odds of a dog to a cat (poisoned)': '5745 to 1', "
 "'odds increase (poisoned vs clean)': '0.39 times', 'odds increase yes/no': "
 'False}\n')
("Image 2 results: {'clean_prediction': 'a dog', 'poisoned_prediction': 'a "
 "dog', 'C_A_prob_ratio_clean': [1.0, 0.00016868114471435547], "
 "'C_A_prob_ratio_poison': [0.9931640625, 0.00669097900